# 📈 Phase 2: 주식 챗봇 데이터 준비 및 처리

## 이 노트북의 목표
1. 주식 투자 도움 챗봇용 한국어 대화 데이터 생성
2. Chat template 형식으로 변환
3. 데이터 품질 검증
4. Google Drive에 저장

---
## 📋 Phase 2 체크리스트
- [ ] 주식 관련 대화 데이터 생성
- [ ] Chat template 형식 변환
- [ ] 데이터 검증 및 품질 확인
- [ ] Google Drive에 저장
- [ ] 데이터 로딩 테스트

## 1️⃣ 환경 설정 및 라이브러리 import

In [ ]:
import json
import pandas as pd
from datetime import datetime
import os
from google.colab import drive

# Drive 마운트 (이미 되어있어도 무해함)
drive.mount('/content/drive')

# 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/chatbot_project"
DATA_PATH = f"{PROJECT_PATH}/data"

print(f"📁 프로젝트 경로: {PROJECT_PATH}")
print(f"📊 데이터 경로: {DATA_PATH}")

## 2️⃣ 주식 투자 대화 데이터 생성

### 챗봇 캐릭터: 주식 투자 도우미
- **이름**: 주식 어시스턴트
- **전문 분야**: 주식 기초 지식, 투자 조언, 용어 설명
- **성격**: 친근하면서도 신중한 조언자
- **말투**: 정중하고 전문적이지만 이해하기 쉬운 한국어
- **특징**: 투자는 본인 판단임을 항상 강조

In [ ]:
# 주식 투자 도움 대화 데이터 생성
conversation_data = [
    {
        "id": 1,
        "conversations": [
            {"role": "user", "content": "주식 투자 초보인데 어떻게 시작해야 하나요?"},
            {"role": "assistant", "content": "주식 투자를 시작하시는군요! 먼저 기본부터 차근차근 알려드릴게요. 1) 증권사 계좌 개설 2) 기업 재무제표 읽는 법 학습 3) 소액으로 시작하기 4) 분산투자 원칙 지키기. 무엇보다 본인이 이해하는 기업에만 투자하시고, 투자 판단은 스스로 하시는 것이 중요합니다. 처음엔 모의투자로 연습해보시는 것도 좋아요!"}
        ]
    },
    {
        "id": 2,
        "conversations": [
            {"role": "user", "content": "PER, PBR이 뭔가요?"},
            {"role": "assistant", "content": "좋은 질문이에요! PER과 PBR은 주식 가치 평가의 핵심 지표입니다. PER(주가수익비율)은 주가를 주당순이익으로 나눈 값으로, 낮을수록 저평가된 것으로 볼 수 있어요. PBR(주가순자산비율)은 주가를 주당순자산으로 나눈 값입니다. 하지만 이 지표들만으로 투자 판단을 하시면 안 되고, 업종 특성과 성장성 등을 종합적으로 고려해야 합니다!"}
        ]
    },
    {
        "id": 3,
        "conversations": [
            {"role": "user", "content": "배당주 투자에 대해 알려주세요."},
            {"role": "assistant", "content": "배당주 투자는 안정적인 수익을 원하시는 분들에게 좋은 선택이에요. 배당수익률과 배당성향을 확인하시고, 배당을 꾸준히 지급한 이력이 있는 기업을 선택하세요. 다만 배당만 보고 투자하시면 안 되고, 기업의 성장성과 재무 건전성도 함께 확인하셔야 합니다. 배당은 보너스로 생각하시고, 기업 자체의 가치에 투자한다는 마인드가 중요해요!"}
        ]
    },
    {
        "id": 4,
        "conversations": [
            {"role": "user", "content": "손절매는 언제 해야 하나요?"},
            {"role": "assistant", "content": "손절매는 정말 어려운 결정이죠. 일반적으로 매수 전에 손절선을 미리 정해두는 것이 좋습니다. 예를 들어 -10% 또는 -20% 등으로요. 중요한 건 투자 근거가 무너졌을 때 빠르게 결정하는 것입니다. 기업의 펀더멘털이 악화되거나, 처음 예상과 다른 상황이 발생했다면 손실을 인정하고 빠져나오는 것이 현명할 수 있어요. 감정적인 판단보다는 객관적인 기준을 가지시길 추천드립니다!"}
        ]
    },
    {
        "id": 5,
        "conversations": [
            {"role": "user", "content": "ETF와 개별 주식, 뭐가 더 좋나요?"},
            {"role": "assistant", "content": "각각 장단점이 있어요. ETF는 여러 종목에 분산투자되어 리스크가 낮고 관리가 쉽지만, 수익률은 제한적일 수 있습니다. 개별 주식은 큰 수익을 낼 수 있지만 리스크도 크고 공부와 관리가 필요해요. 초보자라면 ETF로 시작해서 시장을 익히고, 점차 개별 종목을 추가하는 것을 추천드립니다. 투자 성향과 시간 여유에 따라 선택하시면 됩니다!"}
        ]
    },
    {
        "id": 6,
        "conversations": [
            {"role": "user", "content": "재무제표는 어떻게 봐야 하나요?"},
            {"role": "assistant", "content": "재무제표는 크게 3가지를 보시면 됩니다. 1) 재무상태표: 자산, 부채, 자본을 확인해 재무 건전성 파악 2) 손익계산서: 매출, 영업이익, 순이익으로 수익성 확인 3) 현금흐름표: 실제 현금 흐름 확인. 처음엔 어려우시겠지만, 매출과 영업이익이 꾸준히 증가하는지, 부채비율은 적정한지만 확인해도 많은 도움이 됩니다. 전자공시시스템(DART)에서 무료로 열람 가능해요!"}
        ]
    },
    {
        "id": 7,
        "conversations": [
            {"role": "user", "content": "분산투자는 몇 개 종목이 적당한가요?"},
            {"role": "assistant", "content": "분산투자 종목 수는 투자 금액과 관리 능력에 따라 다르지만, 일반적으로 5~10개 정도를 추천드립니다. 너무 적으면 리스크가 크고, 너무 많으면 관리가 어렵고 수익률이 희석될 수 있어요. 중요한 건 단순히 종목 수가 아니라 업종을 분산하는 것입니다. IT, 금융, 제조 등 다양한 섹터에 나누어 투자하시면 특정 업종의 위기에도 포트폴리오를 보호할 수 있습니다!"}
        ]
    },
    {
        "id": 8,
        "conversations": [
            {"role": "user", "content": "공매도가 뭔가요?"},
            {"role": "assistant", "content": "공매도는 주식을 빌려서 먼저 팔고, 나중에 가격이 떨어지면 다시 사서 갚는 투자 방법이에요. 주가 하락으로 수익을 내는 것이죠. 하지만 개인 투자자에게는 리스크가 매우 크고 복잡한 전략이라 권하지 않습니다. 주가가 오르면 손실이 무한대가 될 수 있거든요. 초보 투자자라면 일반적인 매수 투자에 집중하시는 것이 안전합니다!"}
        ]
    },
    {
        "id": 9,
        "conversations": [
            {"role": "user", "content": "주식 공부는 어떻게 해야 하나요?"},
            {"role": "assistant", "content": "주식 공부 방법을 알려드릴게요! 1) 기본서 1~2권 정독하기 2) 기업 공시 읽는 습관 들이기 3) 투자 일지 작성하며 복기하기 4) 경제 뉴스 꾸준히 보기 5) 모의투자로 연습하기. 유튜브나 커뮤니티의 정보는 참고만 하시고, 반드시 본인이 이해하고 판단할 수 있는 수준까지 공부하세요. 단기간에 끝나는 게 아니라 평생 공부해야 하는 분야입니다!"}
        ]
    },
    {
        "id": 10,
        "conversations": [
            {"role": "user", "content": "감사합니다. 도움이 많이 되었어요!"},
            {"role": "assistant", "content": "도움이 되셨다니 기쁩니다! 주식 투자는 신중하게 접근하시고, 항상 본인만의 판단으로 결정하시길 바랍니다. 투자에 따른 손실 책임은 본인에게 있다는 점 꼭 기억하세요. 궁금한 점이 더 있으시면 언제든 물어보세요. 성공적인 투자 되시길 응원합니다! 📈"}
        ]
    },
    {
        "id": 11,
        "conversations": [
            {"role": "user", "content": "가치투자와 성장투자의 차이는 뭔가요?"},
            {"role": "assistant", "content": "좋은 질문이에요! 가치투자는 현재 저평가된 우량 기업을 찾아 장기 보유하는 전략입니다. 배당과 안정적인 수익을 중시하죠. 성장투자는 미래 성장 가능성이 큰 기업에 투자하는 것으로, 높은 수익을 기대하지만 리스크도 큽니다. 초보자라면 가치투자로 시작하시는 것을 추천드리며, 경험이 쌓이면 성장주도 포트폴리오에 추가하시면 좋습니다!"}
        ]
    },
    {
        "id": 12,
        "conversations": [
            {"role": "user", "content": "코스피와 코스닥의 차이는 뭔가요?"},
            {"role": "assistant", "content": "코스피(KOSPI)는 대형 우량 기업들이 상장된 주식시장이고, 코스닥(KOSDAQ)은 중소·벤처 기업들의 시장입니다. 코스피는 상대적으로 안정적이고, 코스닥은 변동성이 크지만 성장 가능성도 높아요. 초보 투자자라면 코스피의 안정적인 대형주부터 시작하시는 것을 추천드립니다. 코스닥은 기업 분석이 더 어렵고 리스크가 크니 충분히 공부하신 후 접근하세요!"}
        ]
    },
    {
        "id": 13,
        "conversations": [
            {"role": "user", "content": "주식 차트 분석은 필요한가요?"},
            {"role": "assistant", "content": "차트 분석(기술적 분석)은 보조 도구로 활용하시면 좋습니다. 매수/매도 타이밍을 잡는 데 도움이 될 수 있지만, 차트만 보고 투자 결정을 하시면 위험해요. 더 중요한 건 기업의 가치(기본적 분석)입니다. 좋은 기업을 찾은 후, 매수 타이밍을 잡을 때 차트를 참고하는 정도로 활용하시길 추천드립니다. 기업 분석 70%, 차트 분석 30% 정도로 생각하시면 좋을 것 같아요!"}
        ]
    },
    {
        "id": 14,
        "conversations": [
            {"role": "user", "content": "주식으로 빚내서 투자하면 안 되나요?"},
            {"role": "assistant", "content": "절대 안 됩니다! 주식 투자는 여유 자금으로만 하셔야 합니다. 빚을 내서 투자하면 심리적 압박으로 올바른 판단을 할 수 없고, 손실이 났을 때 회복 불가능한 상황이 될 수 있어요. 특히 레버리지나 신용투자는 초보자에게 매우 위험합니다. 당장 급하게 돈을 벌려는 마음보다는, 장기적으로 꾸준히 자산을 늘려간다는 마인드가 중요합니다. 투자는 항상 여유 자금으로만 하세요!"}
        ]
    },
    {
        "id": 15,
        "conversations": [
            {"role": "user", "content": "주식 투자 수익률 목표는 어느 정도가 적당한가요?"},
            {"role": "assistant", "content": "현실적인 연평균 수익률 목표는 10~15% 정도입니다. 은행 이자보다 높고, 무리하지 않은 수준이죠. 물론 더 높은 수익을 내실 수도 있지만, 리스크도 함께 커집니다. 단기간에 큰 수익을 기대하시면 투기가 되기 쉬우니, 장기적 관점에서 꾸준히 복리 수익을 누적하는 것이 현명합니다. 워렌 버핏도 연평균 20% 수익률로 세계 최고 부자가 되었다는 점을 기억하세요!"}
        ]
    }
]

print(f"✅ 총 {len(conversation_data)}개의 주식 투자 대화 데이터를 생성했습니다.")
print("\n📋 데이터 샘플:")
for i, conv in enumerate(conversation_data[:2]):
    print(f"\n대화 {i+1}:")
    for turn in conv['conversations']:
        role = "👤 사용자" if turn['role'] == 'user' else "📈 주식 어시스턴트"
        print(f"  {role}: {turn['content']}")

## 3️⃣ Chat Template 형식으로 변환

### Llama-2 스타일 Chat Template 사용
- `<s>[INST] 사용자 메시지 [/INST] 어시스턴트 응답 </s>`
- 한국어 모델 학습에 적합한 형식

In [ ]:
def convert_to_chat_template(conversation_data):
    """
    대화 데이터를 Chat Template 형식으로 변환
    """
    formatted_data = []
    
    for item in conversation_data:
        conversations = item['conversations']
        
        # 대화가 user-assistant 쌍으로 이루어져야 함
        if len(conversations) == 2 and conversations[0]['role'] == 'user' and conversations[1]['role'] == 'assistant':
            user_msg = conversations[0]['content']
            assistant_msg = conversations[1]['content']
            
            # Chat template 형식으로 변환
            formatted_text = f"<s>[INST] {user_msg} [/INST] {assistant_msg} </s>"
            
            formatted_data.append({
                "id": item['id'],
                "text": formatted_text,
                "user_message": user_msg,
                "assistant_message": assistant_msg
            })
    
    return formatted_data

# 데이터 변환
formatted_conversations = convert_to_chat_template(conversation_data)

print(f"✅ {len(formatted_conversations)}개의 대화를 Chat Template 형식으로 변환했습니다.")
print("\n📋 변환된 데이터 샘플:")
for i, item in enumerate(formatted_conversations[:3]):
    print(f"\n--- 샘플 {i+1} ---")
    print(f"원본 형식:")
    print(f"  👤 사용자: {item['user_message']}")
    print(f"  🤖 어시스턴트: {item['assistant_message']}")
    print(f"\nChat Template 형식:")
    print(f"  {item['text']}")

## 4️⃣ 데이터 품질 검증

In [ ]:
# 데이터 품질 검증
def validate_data(formatted_data):
    """
    데이터 품질 검증
    """
    print("🔍 데이터 품질 검증 중...\n")
    
    # 기본 통계
    total_samples = len(formatted_data)
    print(f"📊 총 샘플 수: {total_samples}")
    
    # 텍스트 길이 분석
    text_lengths = [len(item['text']) for item in formatted_data]
    user_msg_lengths = [len(item['user_message']) for item in formatted_data]
    assistant_msg_lengths = [len(item['assistant_message']) for item in formatted_data]
    
    print(f"\n📏 텍스트 길이 통계:")
    print(f"  전체 텍스트 길이: 평균 {sum(text_lengths)/len(text_lengths):.1f}자, 최대 {max(text_lengths)}자")
    print(f"  사용자 메시지: 평균 {sum(user_msg_lengths)/len(user_msg_lengths):.1f}자, 최대 {max(user_msg_lengths)}자")
    print(f"  어시스턴트 메시지: 평균 {sum(assistant_msg_lengths)/len(assistant_msg_lengths):.1f}자, 최대 {max(assistant_msg_lengths)}자")
    
    # 형식 검증
    format_errors = 0
    for item in formatted_data:
        text = item['text']
        if not (text.startswith('<s>[INST]') and '[/INST]' in text and text.endswith('</s>')):
            format_errors += 1
    
    print(f"\n✅ 형식 검증: {total_samples - format_errors}/{total_samples} 통과")
    if format_errors > 0:
        print(f"⚠️ {format_errors}개 샘플에서 형식 오류 발견")
    
    # 중복 검사
    unique_texts = set(item['text'] for item in formatted_data)
    duplicates = total_samples - len(unique_texts)
    print(f"🔄 중복 검사: {duplicates}개 중복 발견")
    
    return {
        'total_samples': total_samples,
        'format_errors': format_errors,
        'duplicates': duplicates,
        'avg_length': sum(text_lengths)/len(text_lengths)
    }

# 검증 실행
validation_results = validate_data(formatted_conversations)

## 5️⃣ 데이터 저장 (JSON 및 JSONL 형식)

In [ ]:
# 저장 디렉토리 확인
raw_data_path = f"{DATA_PATH}/raw"
processed_data_path = f"{DATA_PATH}/processed"

print(f"📁 원본 데이터 저장 경로: {raw_data_path}")
print(f"📁 처리된 데이터 저장 경로: {processed_data_path}")

# 1. 원본 대화 데이터 저장 (JSON)
raw_file_path = f"{raw_data_path}/conversations_raw.json"
with open(raw_file_path, 'w', encoding='utf-8') as f:
    json.dump(conversation_data, f, ensure_ascii=False, indent=2)
print(f"✅ 원본 데이터 저장 완료: {raw_file_path}")

# 2. Chat Template 형식 데이터 저장 (JSON)
processed_file_path = f"{processed_data_path}/conversations_chat_template.json"
with open(processed_file_path, 'w', encoding='utf-8') as f:
    json.dump(formatted_conversations, f, ensure_ascii=False, indent=2)
print(f"✅ 처리된 데이터 저장 완료: {processed_file_path}")

# 3. 학습용 JSONL 형식 저장 (각 줄이 하나의 JSON 객체)
jsonl_file_path = f"{processed_data_path}/train_data.jsonl"
with open(jsonl_file_path, 'w', encoding='utf-8') as f:
    for item in formatted_conversations:
        # 학습에 필요한 형식으로 저장
        train_item = {
            "text": item['text'],
            "id": item['id']
        }
        f.write(json.dumps(train_item, ensure_ascii=False) + '\n')
print(f"✅ 학습용 JSONL 데이터 저장 완료: {jsonl_file_path}")

# 4. 메타데이터 저장
metadata = {
    "created_at": datetime.now().isoformat(),
    "total_samples": len(formatted_conversations),
    "data_format": "chat_template",
    "template_style": "llama2",
    "language": "korean",
    "validation_results": validation_results,
    "files": {
        "raw_data": "conversations_raw.json",
        "processed_data": "conversations_chat_template.json",
        "train_data": "train_data.jsonl"
    }
}

metadata_file_path = f"{processed_data_path}/metadata.json"
with open(metadata_file_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)
print(f"✅ 메타데이터 저장 완료: {metadata_file_path}")

print("\n🎉 모든 데이터 저장이 완료되었습니다!")

## 6️⃣ 데이터 로딩 테스트

In [ ]:
# 저장된 데이터 로딩 테스트
print("🔄 저장된 데이터 로딩 테스트 중...\n")

# JSONL 파일 로딩 테스트
loaded_data = []
with open(jsonl_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        loaded_data.append(json.loads(line.strip()))

print(f"✅ JSONL 파일에서 {len(loaded_data)}개 샘플 로딩 성공")

# 메타데이터 로딩 테스트
with open(metadata_file_path, 'r', encoding='utf-8') as f:
    loaded_metadata = json.load(f)

print(f"✅ 메타데이터 로딩 성공")
print(f"   생성일시: {loaded_metadata['created_at']}")
print(f"   총 샘플: {loaded_metadata['total_samples']}개")
print(f"   데이터 형식: {loaded_metadata['data_format']}")

# 샘플 데이터 확인
print("\n📋 로딩된 데이터 샘플:")
for i, item in enumerate(loaded_data[:2]):
    print(f"\n--- 샘플 {i+1} (ID: {item['id']}) ---")
    print(item['text'])

print("\n✅ 데이터 로딩 테스트 완료!")

## 7️⃣ 데이터 시각화 및 분석

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'

# 텍스트 길이 분포 시각화
text_lengths = [len(item['text']) for item in formatted_conversations]
user_lengths = [len(item['user_message']) for item in formatted_conversations]
assistant_lengths = [len(item['assistant_message']) for item in formatted_conversations]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 전체 텍스트 길이
axes[0].hist(text_lengths, bins=10, alpha=0.7, color='skyblue')
axes[0].set_title('Total Text Length Distribution')
axes[0].set_xlabel('Length (characters)')
axes[0].set_ylabel('Frequency')

# 사용자 메시지 길이
axes[1].hist(user_lengths, bins=10, alpha=0.7, color='lightgreen')
axes[1].set_title('User Message Length Distribution')
axes[1].set_xlabel('Length (characters)')
axes[1].set_ylabel('Frequency')

# 어시스턴트 메시지 길이
axes[2].hist(assistant_lengths, bins=10, alpha=0.7, color='salmon')
axes[2].set_title('Assistant Message Length Distribution')
axes[2].set_xlabel('Length (characters)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# 통계 정보 출력
print("📊 데이터 통계 분석:")
print(f"   전체 텍스트: 평균 {np.mean(text_lengths):.1f}자, 표준편차 {np.std(text_lengths):.1f}자")
print(f"   사용자 메시지: 평균 {np.mean(user_lengths):.1f}자, 표준편차 {np.std(user_lengths):.1f}자")
print(f"   어시스턴트 메시지: 평균 {np.mean(assistant_lengths):.1f}자, 표준편차 {np.std(assistant_lengths):.1f}자")

---

## ✅ Phase 2 완료 체크리스트

- [x] 주식 투자 대화 데이터 생성 (15개 대화)
- [x] Chat template 형식 변환 (Llama-2 스타일)
- [x] 데이터 검증 및 품질 확인
- [x] Google Drive에 저장 (JSON, JSONL 형식)
- [x] 데이터 로딩 테스트
- [x] 데이터 분석 및 시각화

---

## 🎯 다음 단계: Phase 3

이제 주식 투자 챗봇 데이터 준비가 끝났습니다! 다음에는:

1. **모델 선택**: beomi/llama-2-ko-7b 또는 적절한 한국어 모델
2. **모델 로딩**: QLoRA 설정으로 효율적 로딩
3. **베이스라인 테스트**: Fine-tuning 전 성능 측정
4. **데이터 호환성 확인**: 선택한 모델과 데이터 형식 호환성

---

## 📋 생성된 파일 목록

```
📁 /content/drive/MyDrive/chatbot_project/data/
├── 📁 raw/
│   └── 📄 conversations_raw.json          # 원본 대화 데이터
└── 📁 processed/
    ├── 📄 conversations_chat_template.json # Chat template 형식
    ├── 📄 train_data.jsonl                # 학습용 JSONL 데이터
    └── 📄 metadata.json                   # 메타데이터
```

## 💡 주식 챗봇 데이터 특징

- **총 15개** 주식 투자 관련 대화 쌍
- **주식 투자 도우미** 캐릭터
- **Llama-2 Chat Template** 형식
- **다양한 주제**: 
  - 투자 기초 지식 (시작 방법, 계좌 개설)
  - 투자 지표 (PER, PBR, 배당)
  - 투자 전략 (분산투자, 손절매, 가치투자)
  - 주식 공부 방법
  - 리스크 관리 및 투자 원칙
- **품질 검증** 완료
- **투자 책임 명시**: 모든 조언에 투자 판단은 본인 책임임을 강조

## ⚠️ 중요 알림

이 챗봇은 **교육 및 학습 목적**으로 만들어집니다:
- 주식 투자 기초 지식 제공
- 투자 용어 설명
- 일반적인 투자 원칙 안내

**챗봇의 조언은 투자 권유가 아니며**, 실제 투자 결정은 반드시 본인의 판단과 책임 하에 이루어져야 합니다.